In [5]:
# Requisitos:
# pip install langchain-openai langchain-core langchain langgraph pydantic requests numpy python-dotenv

import os, time, json, requests, numpy as np
from typing import Literal, TypedDict, Dict, List
from dotenv import load_dotenv; load_dotenv()

from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# ----------------------------
# Constantes y sesión HTTP
# ----------------------------
SYMBOL = "BTCUSDT"

API_KEY = os.getenv("BINANCE_API_KEY", "")
API_SECRET = os.getenv("BINANCE_API_SECRET", "")

SESSION = requests.Session()
# Header real de Binance + UA (aunque sea público)
if API_KEY:
    SESSION.headers.update({"X-MBX-APIKEY": API_KEY})
SESSION.headers.update({"User-Agent": "Mozilla/5.0"})

BINANCE_HOSTS = [
    os.getenv("BINANCE_BASE", "https://api.binance.com"),
    "https://api1.binance.com",
    "https://api2.binance.com",
    "https://api3.binance.com",
    "https://data-api.binance.vision",
]

def _get_with_fallback(path: str, params: dict, timeout=12):
    last_err = None
    for base in BINANCE_HOSTS:
        url = f"{base}{path}"
        try:
            r = SESSION.get(url, params=params, timeout=timeout)
            if r.status_code >= 400 and (r.status_code in (451, 403, 429) or 500 <= r.status_code < 600):
                last_err = requests.HTTPError(f"{r.status_code} for {url}", response=r)
                continue
            r.raise_for_status()
            return r.json(), base
        except (requests.ConnectionError, requests.Timeout, requests.HTTPError) as e:
            last_err = e
            continue
    raise last_err or RuntimeError("Fallo de red al consultar Binance")

# ----------------------------
# Utils: klines + EMA + señal
# ----------------------------
def fetch_klines(symbol: str, interval: str, limit: int = 200):
    data, _ = _get_with_fallback("/api/v3/klines", {"symbol": symbol, "interval": interval, "limit": limit})
    closes = [float(x[4]) for x in data]
    times = [int(x[0]) for x in data]
    return times, closes

def ema(series: List[float], period: int) -> np.ndarray:
    arr = np.array(series, dtype=float)
    k = 2 / (period + 1)
    out = np.zeros_like(arr)
    out[0] = arr[0]
    for i in range(1, len(arr)):
        out[i] = arr[i] * k + out[i-1] * (1 - k)
    return out

def crossover_signal(closes: List[float]) -> Literal["bull","bear","none"]:
    if len(closes) < 50:
        return "none"
    e9, e21 = ema(closes, 9), ema(closes, 21)
    prev, curr = e9[-2] - e21[-2], e9[-1] - e21[-1]
    eps = max(1e-6, 0.0001 * closes[-1])
    if prev < -eps and curr > eps:
        return "bull"
    if prev >  eps and curr < -eps:
        return "bear"
    return "none"

# ----------------------------
# Tools para el agente
# ----------------------------
@tool
def get_btc_signal(interval: Literal["1h","4h","1d"]="1h", limit: int = 200) -> str:
    """Devuelve 'interval:signal @ ts UTC (close=...)' usando cruce EMA9/EMA21."""
    times, closes = fetch_klines(SYMBOL, interval, limit)
    sig = crossover_signal(closes)
    ts = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(times[-1]/1000))
    return f"{interval}:{sig} @ {ts} UTC (close={closes[-1]:.2f})"

TOOLS = [get_btc_signal]

# ----------------------------
# LLM para explicación
# ----------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Eres un agente de trading profesional con herramientas:\n{tools}\n\n"
     "Regla:\n- 3/3 bull -> COMPRAR\n- 3/3 bear -> VENDER\n- 2/3 coinciden -> OBSERVAR\n- otro -> SIN SEÑAL\n"
     "Responde breve y con disclaimer."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent_runnable = create_tool_calling_agent(llm, TOOLS, prompt)
agent = AgentExecutor(agent=agent_runnable, tools=TOOLS, verbose=False)

# ----------------------------
# Estado y nodos del grafo
# ----------------------------
class GraphState(TypedDict):
    input: str
    tool_logs: List[str]
    signals: Dict[str, str]
    decision: str
    explanation: str
    loop: int              # contador de ciclos realizados
    max_loops: int         # 0 = infinito; >0 = límite
    sleep_seconds: int     # 300 = 5 minutos

def fetch_node(state: GraphState) -> GraphState:
    tool_logs, signals = [], {}
    for itv in ["1h", "4h", "1d"]:
        res = get_btc_signal.invoke({"interval": itv})
        tool_logs.append(res)
        tag, sig = res.split(":")[0], res.split(":")[1].split(" ")[0]
        signals[tag] = sig
    return {**state, "tool_logs": tool_logs, "signals": signals}

def decide_node(state: GraphState) -> GraphState:
    signals = state["signals"]
    vals = list(signals.values())
    if all(v == "bull" for v in vals):
        decision = "COMPRAR"
    elif all(v == "bear" for v in vals):
        decision = "VENDER"
    else:
        decision = "OBSERVAR" if (vals.count("bull")==2 or vals.count("bear")==2) else "SIN SEÑAL"

    expl_prompt = ChatPromptTemplate.from_messages([
        ("system", "Resume señales y explica la decisión en 2-3 líneas. Incluye disclaimer."),
        ("human", "Señales: {signals_json}. Decisión: {decision}.")
    ])
    explanation = (expl_prompt | llm).invoke({
        "signals_json": json.dumps(signals, ensure_ascii=False),
        "decision": decision
    }).content

    return {**state, "decision": decision, "explanation": explanation}

def notify_node(state: GraphState) -> GraphState:
    # Placeholder: acá podrías enviar Slack/Telegram/email.
    if state["decision"] in ("COMPRAR", "VENDER"):
        print(f"[NOTIFY] {state['decision']} | {state['explanation']}")
    else:
        print(f"[INFO] {state['decision']} | {state['explanation']}")
    return state

def sleep_node(state: GraphState) -> GraphState:
    secs = state.get("sleep_seconds", 300)
    print(f"[SLEEP] Esperando {secs} segundos...")
    time.sleep(secs)
    loop = state.get("loop", 0) + 1
    return {**state, "loop": loop}

def router_after_sleep(state: GraphState) -> str:
    """Decide si seguimos ciclando o terminamos."""
    max_loops = state.get("max_loops", 0)
    loop = state.get("loop", 0)
    if max_loops > 0 and loop >= max_loops:
        return "stop"
    return "continue"

# ----------------------------
# Construcción del grafo
# ----------------------------
graph = StateGraph(GraphState)
graph.add_node("fetch", fetch_node)
graph.add_node("decide", decide_node)
graph.add_node("notify", notify_node)
graph.add_node("sleep", sleep_node)

graph.set_entry_point("fetch")
graph.add_edge("fetch", "decide")
graph.add_edge("decide", "notify")
graph.add_edge("notify", "sleep")
graph.add_conditional_edges("sleep", router_after_sleep, {"continue": "fetch", "stop": END})

memory = MemorySaver()
app = graph.compile(checkpointer=memory)

# ----------------------------
# Main
# ----------------------------
if __name__ == "__main__":
    initial = {
        "input": "Evaluar señales BTC (1h/4h/1d)",
        "tool_logs": [],
        "signals": {},
        "decision": "",
        "explanation": "",
        "loop": 0,
        "max_loops": 0,       # 0 = bucle infinito; poné p.ej. 3 para probar 3 ciclos
        "sleep_seconds": 300  # 5 minutos
    }
    result = app.invoke(initial, config={"configurable": {"thread_id": "btc-signals-loop"}})


[INFO] SIN SEÑAL | La decisión es "SIN SEÑAL" porque no se identificaron indicios de movimiento en ninguna de las temporalidades analizadas (1 hora, 4 horas y 1 día). Esto sugiere que no hay oportunidades claras para operar en este momento. 

Disclaimer: Esta información no constituye asesoramiento financiero y se debe considerar en el contexto de un análisis más amplio.
[SLEEP] Esperando 300 segundos...


KeyboardInterrupt: 